# 1. Introduction
Algorithmic trading refers to the use of computer programs and quantitative models to execute trades in financial markets automatically. By leveraging data science techniques, algorithmic trading aims to identify profitable trading opportunities based on statistical patterns, historical data, and real-time market signals. This project explores two key trading strategies—**Momentum Trading** and **Mean Reversion**—applied to stock price data of Apple Inc. (AAPL) fetched from Yahoo Finance.

### Data Science Aspects in Algorithmic Trading
- **Time Series Analysis:** Stock price movements are analyzed using historical data to identify trends.
- **Technical Indicators:** Momentum (RSI, percentage change) and mean reversion (Z-score, Bollinger Bands) are used to generate buy/sell signals.
- **Visualization:** Interactive plots (using Plotly) help in understanding price movements and strategy performance.
- **Automation:** Python scripts fetch real-time data, compute indicators, and generate trading signals without manual intervention.

This project demonstrates how data science techniques can be applied to financial markets for systematic trading decisions.

In [ ]:
pip install yfinance ta

In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import yfinance as yf
import ta

# 2. Data Collection & Preprocessing

### Data Collection
The project uses **Yahoo Finance (yfinance)** to fetch historical stock data for AAPL with the following parameters:
- **Time Period:** 1 year
- **Interval:** Daily (1d)
- **Features:** Open, High, Low, Close (OHLC), Volume

**Error Handling:**
- Checks for empty data returns.
- Raises exceptions if data fetching fails.

In [ ]:
try:
    stock = yf.Ticker("AAPL")
    data = stock.history(period="1y", interval="1d")
    if data.empty:
        raise ValueError("No data returned from Yahoo Finance")
except Exception as e:
    print(f"Error fetching data: {e}")
    raise

print(data.head())

### Preprocessing Steps
1. **Momentum Strategy:**
   - Computes 1-day and 5-day percentage price changes (`Momentum_1D`, `Momentum_5D`).
   - Calculates **Relative Strength Index (RSI)** using a 14-day window.
   - Generates signals:
     - **Buy (1):** If `Momentum_5D > 0` and `30 < RSI < 70`
     - **Sell (-1):** If `Momentum_5D < 0` and `RSI > 70`
     - **Neutral (0):** Otherwise

In [ ]:
data['Momentum_1D'] = data['Close'].pct_change(periods=1)
data['Momentum_5D'] = data['Close'].pct_change(periods=5)
data['RSI'] = ta.momentum.RSIIndicator(data['Close'], window=14).rsi()

data['Signal'] = np.where(
    (data['Momentum_5D'] > 0) & (data['RSI'] > 30) & (data['RSI'] < 70),
    1,
    np.where(
        (data['Momentum_5D'] < 0) & (data['RSI'] > 70),
        -1,
        0
    )
)

### Visualization
- **Momentum Strategy:**
  - Price chart with buy/sell markers.
  - 5-day momentum trendline.

In [ ]:
figure = make_subplots(
    rows=2, cols=1,
    shared_xaxes=True,
    subplot_titles=('Price with Momentum Signals', 'Momentum Indicator')
)

figure.add_trace(
    go.Scatter(
        x=data.index,
        y=data['Close'],
        name='Close Price',
        line=dict(color='#1f77b4')
    ),
    row=1, col=1
)

figure.add_trace(
    go.Scatter(
        x=data.index,
        y=data['Momentum_5D'],
        name='5-Day Momentum',
        line=dict(color='purple')
    ),
    row=2, col=1
)

figure.add_hline(
    y=0,
    line_dash="solid",
    line_color="black",
    row=2, col=1
)

figure.add_trace(go.Scatter(
    x=data[data['Signal'] == 1].index,
    y=data[data['Signal'] == 1]['Close'],
    mode='markers',
    name='Buy',
    marker=dict(
        color='green',
        symbol='triangle-up',
        size=10,
        line=dict(width=1)
    )
), row=1, col=1)

figure.add_trace(go.Scatter(
    x=data[data['Signal'] == -1].index,
    y=data[data['Signal'] == -1]['Close'],
    mode='markers',
    name='Sell',
    marker=dict(
        color='red',
        symbol='triangle-down',
        size=10,
        line=dict(width=1)
    )
), row=1, col=1)

figure.update_layout(
    title='Algorithmic Trading using Momentum Strategy',
    height=800,
    showlegend=True,
    hovermode='x unified'
)

figure.update_yaxes(title_text="Price", row=1, col=1)
figure.update_yaxes(title_text="5-Day Momentum", row=2, col=1)

figure.show()

### Preprocessing Steps
2. **Mean Reversion Strategy:**
   - Computes **20-day Simple Moving Average (SMA)** and **Standard Deviation (STD)**.
   - Defines **Upper and Lower Bands** (SMA ± 1.5 × STD).
   - Calculates **Z-Score** to measure deviations from the mean.
   - Generates signals:
     - **Buy:** If price falls below the lower band (`Z-Score < -1.5`).
     - **Sell:** If price exceeds the upper band (`Z-Score > 1.5`).

In [ ]:
window = 20
data['SMA'] = data['Close'].rolling(window=window).mean()
data['STD'] = data['Close'].rolling(window=window).std()
data['Upper'] = data['SMA'] + (data['STD'] * 1.5)
data['Lower'] = data['SMA'] - (data['STD'] * 1.5)

In [ ]:
data['Z-Score'] = (data['Close'] - data['SMA']) / data['STD']

### Visualization
- **Mean Reversion Strategy:**
  - Price chart with SMA and Bollinger Bands.
  - Z-Score plot with thresholds.

In [ ]:
figure = make_subplots(rows=2, cols=1, shared_xaxes=True,
                      subplot_titles=('Price with Mean Reversion Bands', 'Z-Score'))

figure.add_trace(go.Scatter(x=data.index, y=data['Close'], name='Close Price'), row=1, col=1)
figure.add_trace(go.Scatter(x=data.index, y=data['SMA'], name=f'{window}-Day SMA',
                           line=dict(color='orange', width=1)), row=1, col=1)
figure.add_trace(go.Scatter(x=data.index, y=data['Upper'], name='Upper Band',
                           line=dict(color='red', width=1, dash='dash')), row=1, col=1)
figure.add_trace(go.Scatter(x=data.index, y=data['Lower'], name='Lower Band',
                           line=dict(color='green', width=1, dash='dash')), row=1, col=1)

figure.add_trace(go.Scatter(x=data.index, y=data['Z-Score'], name='Z-Score',
                           line=dict(color='purple')), row=2, col=1)
figure.add_hline(y=1.5, line_dash="dot", line_color="red", row=2, col=1)
figure.add_hline(y=-1.5, line_dash="dot", line_color="green", row=2, col=1)
figure.add_hline(y=0, line_dash="solid", line_color="black", row=2, col=1)

figure.add_trace(go.Scatter(
    x=data.loc[data['Z-Score'] < -1.5].index,
    y=data.loc[data['Z-Score'] < -1.5]['Close'],
    mode='markers', name='Buy',
    marker=dict(color='green', symbol='triangle-up', size=10)), row=1, col=1)

figure.add_trace(go.Scatter(
    x=data.loc[data['Z-Score'] > 1.5].index,
    y=data.loc[data['Z-Score'] > 1.5]['Close'],
    mode='markers', name='Sell',
    marker=dict(color='red', symbol='triangle-down', size=10)), row=1, col=1)

figure.update_layout(title='Algorithmic Trading using Mean Reversion Strategy',
                   height=800)
figure.update_yaxes(title_text="Price", row=1, col=1)
figure.update_yaxes(title_text="Z-Score", row=2, col=1)
figure.show()